In [2]:
import numpy as np
from emo_utils import *
from keras.layers import Input, Dense, Activation, LSTM, Bidirectional, RepeatVector, Dot, Concatenate, Embedding
from keras.layers import Conv1D
from keras.optimizers import Adam
from keras.models import Model
from keras.initializers import glorot_uniform

In [3]:
data, labels, max_length =load_data()

In [4]:
Tx=max_length
Tx

66

In [5]:
label_dict, y_hot=labels_to_one_hot(labels)

In [6]:
word_to_vec, Glove_words, word_to_index=load_GloVe()

In [7]:
X_index=data_to_index(data, word_to_index, Glove_words, Tx)

In [8]:
def pretrained_embedding_layer(word_to_vec, word_to_index):
    
    vocab_size=len(word_to_index)+1     ### +1 for unknown words which are not present in the Glove_words
    
    emb_dim= word_to_vec['this'].shape[0]
    
    
    embedding_matrix=np.zeros((vocab_size, emb_dim))
    
    for w, i in word_to_index.items():
        
        embedding_matrix[i]=word_to_vec[w]
    
    embedding_layer= Embedding(input_dim=vocab_size, output_dim=emb_dim, trainable=False)  ## can only be used as 
                                                                                            ### the first layer in a model
    embedding_layer.build((None,))
    
    embedding_layer.set_weights([embedding_matrix])
    
    return embedding_layer
    

In [9]:
repeator=RepeatVector(Tx)
concatenator=Concatenate(axis=-1)
dotor=Dot(axes=1)

In [10]:
def one_step_attention(s_prev, a):
    
    s_prev=repeator(s_prev)
    
    concat=concatenator([s_prev,a])
    
    e=Dense(units=32, activation='tanh')(concat)
    
    e=Dense(units=64, activation='relu')(e)
    
    alphas= Activation("softmax")(e)
    
    context=dotor([alphas, a])
    
    return context


In [11]:
def model(word_to_vec, word_to_index, n_a, n_s, Tx, label_dict_size):
    
    X_input=Input(shape=(Tx,))
    
    embedding_layer= pretrained_embedding_layer(word_to_vec, word_to_index)
    
    X=embedding_layer(X_input)                  ## X is the embeddings of input  
    
    a=Bidirectional(LSTM(units=n_a, return_sequences=True))(X)
    
    a=Conv1D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer=glorot_uniform(seed=0))(a)
    
    s0=Input(shape=(n_s,))
    c0=Input(shape=(n_s,))
    
    context=one_step_attention(s0, a)
    
    s,_,c= LSTM(units=n_s, return_state=True)([context, s0,c0])
    
    out= Dense(units=label_dict_size, activation='softmax')(s)
    
    model=Model(inputs=[X_input, s0, c0], outputs=out)
    
    return model
    

In [12]:
n_a=32
n_s=64
model=model(word_to_vec, word_to_index, n_a=32,n_s=64, Tx=Tx, label_dict_size=len(label_dict))

In [13]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 66)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 66, 100)       40000100    input_1[0][0]                    
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 64)            0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 66, 64)        34048       embedding_1[0][0]                
___________________________________________________________________________________________

In [14]:
opt=Adam(lr=0.02, beta_1=0.9, beta_2=0.999, decay=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
m=len(data)
n_s=64
s0=np.zeros((m, n_s))
c0=np.zeros((m, n_s))

In [16]:
model.fit([X_index,s0,c0], y_hot, epochs=15, batch_size=128)   ### batch_size in the form 2^k

Epoch 1/15
16000/16000 [==============================] - 84s - loss: 1.5893 - acc: 0.3324    
Epoch 2/15
16000/16000 [==============================] - 81s - loss: 1.4548 - acc: 0.4389    
Epoch 3/15
16000/16000 [==============================] - 81s - loss: 1.2419 - acc: 0.5327    
Epoch 4/15
16000/16000 [==============================] - 81s - loss: 1.1949 - acc: 0.5443    
Epoch 5/15
16000/16000 [==============================] - 81s - loss: 1.1576 - acc: 0.5562    
Epoch 6/15
16000/16000 [==============================] - 81s - loss: 1.1160 - acc: 0.5761    
Epoch 7/15
16000/16000 [==============================] - 81s - loss: 1.0837 - acc: 0.5932    
Epoch 8/15
16000/16000 [==============================] - 81s - loss: 1.0535 - acc: 0.6071    
Epoch 9/15
16000/16000 [==============================] - 81s - loss: 1.0304 - acc: 0.6168    
Epoch 10/15
16000/16000 [==============================] - 82s - loss: 1.0091 - acc: 0.6226    
Epoch 11/15
16000/16000 [========================

## Val set

In [17]:
val_data, val_label, val_max_length=load_val_data()

In [18]:
val_max_length

61

In [19]:
X_val_index=data_to_index(val_data, word_to_index, Glove_words, Tx)

In [20]:
_, y_val_hot=labels_to_one_hot(val_label)

In [21]:
m=len(val_data)
n_s=64
s0=np.zeros((m, n_s))
c0=np.zeros((m, n_s))

In [22]:
loss, acc= model.evaluate([X_val_index, s0,c0], y_val_hot)

1984/2000 [============================>.] - ETA: 0s

In [25]:
print("loss: %f"%loss)
print("acc: %f" % acc)

loss: 0.929652
acc: 0.653500


In [33]:
model.get_weights()

[array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [ 0.38472  ,  0.49351  ,  0.49096  , ...,  0.026263 ,  0.39052  ,
          0.52217  ],
        [ 0.22657  ,  0.64651  ,  0.84828  , ...,  0.54712  ,  0.7697   ,
          0.35075  ],
        ...,
        [ 0.14335  ,  0.557    , -0.68806  , ...,  0.10501  , -0.49575  ,
          0.39039  ],
        [-0.036419 , -0.63433  , -0.26185  , ...,  0.25043  ,  0.21037  ,
          0.75933  ],
        [ 0.32008  ,  0.21479  , -0.036466 , ...,  0.088318 ,  0.11623  ,
          0.0020262]], dtype=float32),
 array([[-0.27879798,  0.03305028, -0.1328084 , ..., -0.08771858,
          0.09036322,  0.05531954],
        [ 0.01349655,  0.07120806,  0.18319121, ..., -0.02620465,
          0.07661589, -0.15081999],
        [ 0.0725911 , -0.12794329, -0.00501771, ..., -0.11778858,
         -0.08930881, -0.0538554 ],
        ...,
        [-0.05277747,  0.05490657,  0.00410157, ...,  0.14356758,
         